### Learning a Latent Space of Multitrack Measures

MusicVAE learns a latent space of musical sequences. We apply the MusicVAE framework to single measures of multi-instrument General MIDI, a symbolic music representation that uses a standard set of 128 instrument sounds.

The models in this notebook are capable of encoding and decoding single measures of up to 8 tracks, optionally conditioned on an underlying chord. Encoding transforms a single measure into a vector in a latent space, and decoding transforms a latent vector back into a measure. Both encoding and decoding are performed hierarchically, with one level operating on tracks and another operating on the notes (and choice of instrument) in each track.

#### Env Setup

In [1]:
!gsutil -q -m cp gs://download.magenta.tensorflow.org/models/music_vae/multitrack/* /content/
!gsutil -q -m cp gs://download.magenta.tensorflow.org/soundfonts/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2 /content/

!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -qU magenta pyfluidsynth pretty_midi

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.6MB 1.3MB/s 
     |████████████████████████████████| 5.6MB 7.3MB/s 
     |████████████████████████████████| 256kB 24.7MB/s 
     |████████████████████████████████| 215kB 20.8MB/s 
     |████████████████████████████████| 204kB 21.4MB/s 
     |████████████████████████████████| 2.3MB 23.9MB/s 
     |████████████████████████████████| 358kB 31.2MB/s 
     |████████████████████████████████| 1.5MB 28.8MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 71kB 7.

In [2]:
import ctypes.util
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return ctypes.util.find_library(lib)
ctypes.util.find_library = proxy_find_library

In [3]:
import os

import numpy as np
import magenta.music as mm
import tensorflow.compat.v1 as tf

from google.colab import files
from magenta.models.music_vae import configs
from magenta.music.sequences_lib import concatenate_sequences
from magenta.models.music_vae.trained_model import TrainedModel

In [4]:
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
BATCH_SIZE = 4
Z_SIZE = 512
TOTAL_STEPS = 512
BAR_SECONDS = 2.0
CHORD_DEPTH = 49

SAMPLE_RATE = 44100
SF2_PATH = '/content/SGM-v2.01-Sal-Guit-Bass-V1.3.sf2'

In [6]:
# Play sequence using SoundFont.
def play(note_sequences):
  if not isinstance(note_sequences, list):
    note_sequences = [note_sequences]
  for ns in note_sequences:
    mm.play_sequence(ns, synth = mm.fluidsynth, sf2_path = SF2_PATH)
  
# Spherical linear interpolation.
def slerp(p0, p1, t):
  """Spherical linear interpolation."""
  omega = np.arccos(np.dot(np.squeeze(p0/np.linalg.norm(p0)), np.squeeze(p1/np.linalg.norm(p1))))
  so = np.sin(omega)
  return np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1

# Download sequence.
def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

# Chord encoding tensor.
def chord_encoding(chord):
  index = mm.TriadChordOneHotEncoding().encode_event(chord)
  c = np.zeros([TOTAL_STEPS, CHORD_DEPTH])
  c[0,0] = 1.0
  c[1:,index] = 1.0
  return c

# Trim sequences to exactly one bar.
def trim_sequences(seqs, num_seconds = BAR_SECONDS):
  for i in range(len(seqs)):
    seqs[i] = mm.extract_subsequence(seqs[i], 0.0, num_seconds)
    seqs[i].total_time = num_seconds

# Consolidate instrument numbers by MIDI program.
def fix_instruments_for_concatenation(note_sequences):
  instruments = {}
  for i in range(len(note_sequences)):
    for note in note_sequences[i].notes:
      if not note.is_drum:
        if note.program not in instruments:
          if len(instruments) >= 8:
            instruments[note.program] = len(instruments) + 2
          else:
            instruments[note.program] = len(instruments) + 1
        note.instrument = instruments[note.program]
      else:
        note.instrument = 9

#### Chord-Conditioned Model

In [7]:
config = configs.CONFIG_MAP['hier-multiperf_vel_1bar_med_chords']
model = TrainedModel(
    config, batch_size = BATCH_SIZE,
    checkpoint_dir_or_path = '/content/model_chords_fb64.ckpt')

INFO:tensorflow:Building MusicVAE model with HierarchicalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 512, 'z_size': 512, 'free_bits': 0.0, 'max_beta': 1.0, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [512, 512, 512], 'enc_rnn_size': [1024], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Hierarchical Encoder:
  input length: 512
  level lengths: [64, 8]

INFO:tensorflow:Level 0 splits: 8
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [1024]

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
INFO:tensorflow:Level 1 splits: 1
INFO:

In [8]:
### Same Chord, different styles

chord = 'C' 
temperature = 0.2 
seqs = model.sample(n = BATCH_SIZE, length = TOTAL_STEPS, temperature = temperature,
                    c_input = chord_encoding(chord))

trim_sequences(seqs)
play(seqs)

In [9]:
### Same Style, Chord Progression

chord_1 = 'C' 
chord_2 = 'Caug' 
chord_3 = 'Am'
chord_4 = 'E' 
chords = [chord_1, chord_2, chord_3, chord_4]

temperature = 0.2
z = np.random.normal(size = [1, Z_SIZE])
seqs = [
    model.decode(length = TOTAL_STEPS, z = z, temperature = temperature,
                 c_input = chord_encoding(c))[0]
    for c in chords
]

trim_sequences(seqs)
fix_instruments_for_concatenation(seqs)
prog_ns = concatenate_sequences(seqs)

play(prog_ns)
mm.plot_sequence(prog_ns)

In [10]:
### Style Interpolation, Repeating Chord Progression

chord_1 = 'Dm' 
chord_2 = 'F' 
chord_3 = 'Am' 
chord_4 = 'G' 
chords = [chord_1, chord_2, chord_3, chord_4]

num_bars = 32 
temperature = 0.2 

z1 = np.random.normal(size = [Z_SIZE])
z2 = np.random.normal(size = [Z_SIZE])
z = np.array([slerp(z1, z2, t)
              for t in np.linspace(0, 1, num_bars)])

seqs = [
    model.decode(length = TOTAL_STEPS, z = z[i:i+1, :], temperature = temperature,
                 c_input = chord_encoding(chords[i % 4]))[0]
    for i in range(num_bars)
]

trim_sequences(seqs)
fix_instruments_for_concatenation(seqs)
prog_interp_ns = concatenate_sequences(seqs)

play(prog_interp_ns)
mm.plot_sequence(prog_interp_ns)

### Unconditioned Model

In [11]:
config = configs.CONFIG_MAP['hier-multiperf_vel_1bar_med']
model = TrainedModel(
    config, batch_size = BATCH_SIZE,
    checkpoint_dir_or_path = '/content/model_fb256.ckpt')
model._config.data_converter._max_tensors_per_input = None

INFO:tensorflow:Building MusicVAE model with HierarchicalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 512, 'z_size': 512, 'free_bits': 0.0, 'max_beta': 1.0, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [512, 512, 512], 'enc_rnn_size': [1024], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Hierarchical Encoder:
  input length: 512
  level lengths: [64, 8]

INFO:tensorflow:Level 0 splits: 8
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [1024]

INFO:tensorflow:Level 1 splits: 1
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [1024]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 512
  level output le